In [86]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import os
from PIL import Image

In [87]:
batch_size = 32
img_width, img_height = 224, 224

bird_dir = r'C:\Shanmathi\Hackathon_Projects\SIH\sih24\Dataset\Preprocessed\Bird'
drone_dir = r'C:\Shanmathi\Hackathon_Projects\SIH\sih24\Dataset\Preprocessed\Drone'

In [88]:
def load_images_from_directory(directory, label, img_size=(img_width, img_height)):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path).resize(img_size)
            img_array = np.array(img)
            images.append(img_array)
            labels.append(label)
    return images, labels

In [89]:
bird_images, bird_labels = load_images_from_directory(bird_dir, 0)
drone_images, drone_labels = load_images_from_directory(drone_dir, 1)

In [90]:
all_images = np.array(bird_images + drone_images)
all_labels = np.array(bird_labels + drone_labels)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, stratify=all_labels)

In [92]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [93]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
    return image, label

In [94]:
train_dataset = train_dataset.map(augment).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [95]:
base_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [96]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [97]:
class_weights = {0: 3.030625, 1: 0.5987898246480613}

In [98]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=50,
    class_weight=class_weights
)

Epoch 1/50


122/122 ━━━━━━━━━━━━━━━━━━━━ 454s 4s/step - accuracy: 0.9330 - loss: 1.0862 - val_accuracy: 0.8701 - val_loss: 21.0454
Epoch 2/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 435s 4s/step - accuracy: 0.9985 - loss: 0.0218 - val_accuracy: 0.8495 - val_loss: 29.9456
Epoch 3/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 439s 4s/step - accuracy: 1.0000 - loss: 1.0897e-11 - val_accuracy: 0.8495 - val_loss: 30.5094
Epoch 4/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 468s 4s/step - accuracy: 1.0000 - loss: 5.9443e-07 - val_accuracy: 0.8495 - val_loss: 29.9041
Epoch 5/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 445s 4s/step - accuracy: 1.0000 - loss: 5.1145e-21 - val_accuracy: 0.8495 - val_loss: 29.9040
Epoch 6/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 364s 3s/step - accuracy: 1.0000 - loss: 2.2753e-13 - val_accuracy: 0.8495 - val_loss: 29.9040
Epoch 7/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 307s 3s/step - accuracy: 0.9984 - loss: 0.0683 - val_accuracy: 0.9887 - val_loss: 1.0022
Epoch 8/50
122/122 ━━━━━━━━━━━━━━━━━━━━ 848s 7s/step - accuracy: 0.9970 - loss: 0.2596

In [99]:
model.save('vgg19_binary_classification_epoch_50.h5')